# How often does @realDonaldTrump tweet each day?

In [1]:
# https://github.com/dnanhkhoa/nb_black
%load_ext lab_black

In [2]:
import altair as alt
import pandas as pd
import matplotlib as mpl
import numpy as np
import json
import pytz
from datetime import datetime
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [3]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

### Read dump downloaded from trumptweetarchive.com

In [4]:
%run '00-trump-tweets-processing.ipynb'

In [5]:
src = pd.read_csv(
    "/Users/mhustiles/data/github/notebooks/\
trump-tweets/output/realdonaldtrump.csv",
    dtype={"id": str},
    low_memory=False,
)

In [6]:
src["user"] = "realDonaldTrump"

### How many tweets since May 2009?

In [7]:
len(src)

56118

### Clean up dates

In [8]:
src["id"] = src["id"].astype(str)
src["year"] = src["year"].astype(str)
src["month"] = src["month"].astype(str)
src["day"] = src["day"].astype(str)
src["hour"] = src["hour"].astype(str)
src["date"] = pd.to_datetime(src["date"])

### Most popular tweets

In [9]:
src[["id", "date", "text", "favorites"]].sort_values("favorites", ascending=False).head(
    5
)

,id,date,text,favorites
11089,1311892190680014800,2020-10-01,"Tonight, @FLOTUS and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!",1869706
11137,1312233807991496700,2020-10-02,"Going welI, I think! Thank you to all. LOVE!!!",1219870
56088,1325099845045071872,2020-11-07,"I WON THIS ELECTION, BY A LOT!",1188311
56031,1326158760826560512,2020-11-10,WE WILL WIN!,1023598
56086,1325195021339987968,2020-11-07,"71,000,000 Legal Votes. The most EVER for a sitting President!",978022


In [10]:
src[["id", "date", "text", "retweets"]].sort_values("retweets", ascending=False).head(5)

,id,date,text,retweets
11089,1311892190680014800,2020-10-01,"Tonight, @FLOTUS and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!",408866
34864,881503147168071700,2017-07-02,#FraudNewsCNN #FNN https://t.co/WYUnHjjUjg,293109
38552,795954831718498300,2016-11-08,TODAY WE MAKE AMERICA GREAT AGAIN!,281289
28862,474134260149157900,2014-06-04,Are you allowed to impeach a president for gross incompetence?,237674
8421,1267637602724839400,2020-06-01,RT @SpaceX: Liftoff! https://t.co/DRBfdUM7JA,235250


In [11]:
src.to_csv("output/trump-tweets-all.csv", index=False)

### Since he took office

In [12]:
trumppres = pd.DataFrame(src[src.date >= "01/20/2017"]).sort_values(
    "date", ascending=True
)

---

### Devices used for tweets as president

In [13]:
trumppres_devices = (
    trumppres.groupby(["device"])
    .agg({"id": "count"})
    .reset_index()
    .rename(columns={"id": "count"})
)

In [14]:
for i in trumppres_devices:
    trumppres_devices["pct"] = round(
        (trumppres_devices["count"] / trumppres_devices["count"].sum()) * 100, 2
    )

In [15]:
trumppres_devices.sort_values("count", ascending=False).head()

,device,count,pct
7,Twitter for iPhone,24754,97.35
2,Twitter Media Studio,372,1.46
5,Twitter for Android,173,0.68
4,Twitter Web Client,48,0.19
6,Twitter for iPad,38,0.15


### How many not from an iPhone since president? 

In [16]:
trumppres_notiphone = trumppres[trumppres["device"] != "Twitter for iPhone"]

In [17]:
len(trumppres_notiphone)

1154

### Time difference between tweets

In [18]:
trumppres = pd.DataFrame(src[src.date >= "01/20/2017"]).sort_values(
    "date", ascending=True
)

In [19]:
trumppres["diff"] = (trumppres["date"] - trumppres["date"].shift(1)).astype(
    "timedelta64[h]"
)

In [20]:
trumppres.sort_values("date", ascending=False).head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,year,month,day,hour,minute,time,user,diff
0,0,1347569870578266115,"To all of those who have asked, I will not be going to the Inauguration on January 20th.",False,False,Twitter for iPhone,510761,83532,2021-01-08,False,2021,1,8,7,44,07:44:28,realDonaldTrump,0.0
1,1,1347555316863553542,"The 75,000,000 great American Patriots who voted for me, AMERICA FIRST, and MAKE AMERICA GREAT AGAIN, will have a GIANT VOICE long into the future. They will not be disrespected or treated unfairly in any way, shape or form!!!",False,False,Twitter for iPhone,480997,108844,2021-01-08,False,2021,1,8,6,46,06:46:38,realDonaldTrump,24.0
2,2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,629326,154877,2021-01-07,False,2021,1,7,16,10,16:10:24,realDonaldTrump,24.0
12,12,1346818855298072576,"They just happened to find 50,000 ballots late last night. The USA is embarrassed by fools. Our Election Process is worse than that of third world countries!",False,False,Twitter for iPhone,377295,92437,2021-01-06,False,2021,1,6,6,0,06:00:12,realDonaldTrump,0.0
7,7,1346900434540240897,"Mike Pence didn’t have the courage to do what should have been done to protect our Country and our Constitution, giving States a chance to certify a corrected set of facts, not the fraudulent or inaccurate ones which they were asked to previously certify. USA demands the truth!",False,True,Twitter for iPhone,0,0,2021-01-06,False,2021,1,6,11,24,11:24:22,realDonaldTrump,24.0


In [21]:
trumppres["diff"].mean()

1.342339908132937

### Tweets per month, year

In [22]:
trump_years = src.groupby(["year", "month"]).agg({"date": "count"}).reset_index()

In [23]:
trump_years.tail()

,year,month,date
136,2020,6,1112
137,2020,7,834
138,2020,8,1119
139,2020,9,1341
140,2021,1,150


### What did he average each day in retweets, likes and replies

In [24]:
trump_engagements_day = (
    src.groupby(["month", "year"])
    .agg({"id": "size", "retweets": ["sum", "mean"], "favorites": ["sum", "mean"]})
    .round()
    .reset_index()
)

In [25]:
trump_engagements_day.columns = (
    trump_engagements_day.columns.map("_".join).str.strip().str.rstrip("_")
)

In [26]:
trump_engagements_day.tail()

,month,year,id_size,retweets_sum,retweets_mean,favorites_sum,favorites_mean
136,9,2016,287,2151751,7497.0,5663059,19732.0
137,9,2017,307,4012787,13071.0,16392461,53396.0
138,9,2018,375,5094550,13585.0,18775513,50068.0
139,9,2019,788,9396138,11924.0,30182716,38303.0
140,9,2020,1341,21154617,15775.0,57707686,43033.0


In [27]:
chart = (
    alt.Chart((trump_engagements_day.query('year > "2016"')))
    .mark_bar(size=10, opacity=1)
    .encode(
        x=alt.X(
            "month:T",
            title=" ",
            axis=alt.Axis(values=["1", "12"], grid=False, tickCount=1, format="%b"),
        ),
        y=alt.Y(
            "id_size:Q",
            stack=None,
            title=" ",
            axis=alt.Axis(
                gridWidth=0.6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=2,
                format="2,f",
            ),
            scale=alt.Scale(domain=(0, 1500)),
        ),
        facet=alt.Facet("year:N", columns=4, title=" "),
    )
    .properties(width=150, height=100, title="Monthly tweets by @realDonaldTrump")
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient="top", symbolType="stroke"
).configure_legend(symbolType="square")

alt.Chart(...)

In [28]:
chart = (
    alt.Chart((trump_engagements_day.query('year > "2016"')))
    .mark_bar(size=10, opacity=1)
    .encode(
        x=alt.X(
            "month:T",
            title=" ",
            axis=alt.Axis(values=["1", "12"], grid=False, tickCount=1, format="%b"),
        ),
        y=alt.Y(
            "favorites_mean:Q",
            stack=None,
            title=" ",
            axis=alt.Axis(
                gridWidth=0.6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=2,
                format="1,s",
            ),
            scale=alt.Scale(domain=(0, 120000)),
        ),
        facet=alt.Facet("year:N", columns=4, title=" "),
    )
    .properties(width=150, height=100, title="Mean monthly likes of @realDonaldTrump")
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient="top", symbolType="stroke"
).configure_legend(symbolType="square")

alt.Chart(...)

In [29]:
chart = (
    alt.Chart((trump_engagements_day.query('year > "2016"')))
    .mark_bar(size=10, opacity=1)
    .encode(
        x=alt.X(
            "month:T",
            title=" ",
            axis=alt.Axis(values=["1", "7"], grid=False, tickCount=1, format="%B"),
        ),
        y=alt.Y(
            "retweets_mean:Q",
            stack=None,
            title=" ",
            axis=alt.Axis(
                gridWidth=0.6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=2,
                format="1,s",
            ),
            scale=alt.Scale(domain=(0, 50000)),
        ),
        facet=alt.Facet("year:N", columns=5, title=" "),
    )
    .properties(
        width=150, height=100, title="Mean monthly retweets of @realDonaldTrump"
    )
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient="top", symbolType="stroke"
).configure_legend(symbolType="square")

alt.Chart(...)

---

### Dump monthly engagements

In [30]:
df_dt = trumppres.copy()

In [31]:
df_dt = df_dt.set_index("date")

In [32]:
g = df_dt.groupby(pd.Grouper(freq="MS"))

In [33]:
trump_engagements_month = g.mean().reset_index()

In [34]:
trump_engagements_month.tail()

,date,index,favorites,retweets,minute,diff
44,2020-09-01,7721.973154,43033.322893,15775.255034,29.014914,0.536913
45,2020-10-01,4665.328602,71776.046132,20828.058197,29.669269,0.528034
46,2020-11-01,1095.652514,132474.614525,33036.382123,30.557542,0.804469
47,2020-12-01,467.432432,123008.089030,32786.109698,31.157393,1.182830
48,2021-01-01,74.533333,138693.660000,38650.913333,27.833333,1.280000


In [35]:
# trump_engagements_month = (
#     src.groupby(["month", "year"])
#     .agg({"id": "size", "retweets": sum, "favorites": sum})
#     .round()
#     .reset_index()
# )

In [36]:
trump_engagements_month[["date", "favorites", "retweets"]].to_csv(
    "output/trump_engagements_month.csv", index=False
)

---

### Limit data to when Trump took office. How many tweets since then?

In [37]:
trumpall_prez = src[src.date >= "01/20/2017"]
len(trumpall_prez)

25908

In [38]:
trumpall_prez.head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,year,month,day,hour,minute,time,user
0,0,1347569870578266115,"To all of those who have asked, I will not be going to the Inauguration on January 20th.",False,False,Twitter for iPhone,510761,83532,2021-01-08,False,2021,1,8,7,44,07:44:28,realDonaldTrump
1,1,1347555316863553542,"The 75,000,000 great American Patriots who voted for me, AMERICA FIRST, and MAKE AMERICA GREAT AGAIN, will have a GIANT VOICE long into the future. They will not be disrespected or treated unfairly in any way, shape or form!!!",False,False,Twitter for iPhone,480997,108844,2021-01-08,False,2021,1,8,6,46,06:46:38,realDonaldTrump
2,2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,629326,154877,2021-01-07,False,2021,1,7,16,10,16:10:24,realDonaldTrump
3,3,1346954970910707712,These are the things and events that happen when a sacred landslide election victory is so unceremoniously &amp; viciously stripped away from great patriots who have been badly &amp; unfairly treated for so long. Go home with love &amp; in peace. Remember this day forever!,False,True,Twitter for iPhone,0,0,2021-01-06,False,2021,1,6,15,1,15:01:04,realDonaldTrump
4,4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,True,Twitter for iPhone,0,0,2021-01-06,False,2021,1,6,13,17,13:17:24,realDonaldTrump


### Limit data to month Trump announced candidacy. How many tweets since then?

In [39]:
trumpall_running = src[src.date >= "06/01/2015"]
len(trumpall_running)

34292

### Count daily tweets since candidacy, create dataframe with results

In [40]:
trump_tweets_day_candidacy = trumpall_running.groupby(["date"]).size()

In [41]:
trump_tweets_day_candidacy.tail()

date
2021-01-04    13
2021-01-05    27
2021-01-06    13
2021-01-07     1
2021-01-08     2
dtype: int64

In [42]:
trump_tweets_day_candidacy_df = pd.DataFrame(
    {
        "date": trump_tweets_day_candidacy.index,
        "count": trump_tweets_day_candidacy.values,
    }
)

In [43]:
trump_tweets_day_candidacy_df.to_csv("output/trump_tweets_day_candidacy.csv")

### Count daily tweets since inaguration, create dataframe with results

In [44]:
trump_tweets_day_prez = trumpall_prez.groupby(["date"]).size()
trump_tweets_day_prez_df = pd.DataFrame(
    {"date": trump_tweets_day_prez.index, "count": trump_tweets_day_prez.values}
)

In [45]:
trump_tweets_day_prez_df.to_csv("output/trump_tweets_day_prez.csv")

### Count daily tweets from the beginning, create dataframe with results

In [46]:
trump_tweets_day = src.groupby(["date"]).agg("size").reset_index(name="count")
trump_tweets_day.tail(10)

,date,count
3493,2020-12-30,15
3494,2020-12-31,5
3495,2021-01-01,28
3496,2021-01-02,26
3497,2021-01-03,40
3498,2021-01-04,13
3499,2021-01-05,27
3500,2021-01-06,13
3501,2021-01-07,1
3502,2021-01-08,2


In [47]:
trump_tweets_day.to_csv("output/trump_tweets_day.csv")

---

### Sort table for top tweet days during presidency

In [48]:
trump_freq_inauguration = trump_tweets_day_prez_df.sort_values(
    ["count"], ascending=False
)

In [49]:
trump_freq_inauguration.head(5)

,date,count
1225,2020-06-05,198
1199,2020-05-10,124
1049,2019-12-12,122
1354,2020-10-12,117
1311,2020-08-30,109


### Read all from today

In [50]:
today = pd.Timestamp.today().strftime("%Y-%m-%d")

In [51]:
len(today)

10

In [52]:
trump_tweets_today = trumpall_prez[trumpall_prez.date == today]

In [53]:
trump_tweets_today.head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,year,month,day,hour,minute,time,user


---

### RT vs Tweet

In [54]:
rt_tweet = src.groupby(["date", "isRetweet"]).agg("size").reset_index(name="count")
rt_tweet_recent = (
    src[src["date"] > "2019-01-01"]
    .groupby(["date", "isRetweet"])
    .agg("size")
    .reset_index(name="count")
)

In [55]:
rt_pivot = pd.DataFrame(
    pd.pivot_table(
        rt_tweet, values="count", index=["date"], columns=["isRetweet"], aggfunc=np.sum
    )
    .rename(columns={"isRetweet": "id", False: "tweet", True: "retweet"})
    .reset_index()
    .fillna("")
)

In [56]:
rt_pivot.to_csv("output/rt_pivot.csv")

---

### Chart it!

In [57]:
# bars

lines = (
    alt.Chart(
        trump_tweets_day_candidacy_df,
        title="@realDonaldTrump tweet frequency since candidacy announcement",
    )
    .mark_bar(size=2)
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(grid=False, title="", tickCount=6, format=("%b. %Y")),
        ),
        y=alt.Y(
            "count:Q",
            scale=alt.Scale(domain=(0, 200)),
            axis=alt.Axis(
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=6,
                title="Daily tweet counts and mean",
            ),
        ),
    )
)

# rule showing mean

rule = (
    alt.Chart(trump_tweets_day_candidacy_df)
    .mark_rule(color="red")
    .encode(y="mean(count):Q")
)

# rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align="center",
    baseline="middle",
    dx=200,
    dy=10,
    fontWeight="bold",
).encode(text=alt.Text("mean(count):Q", format=".2"))

# go
(lines + rule + text).properties(height=400, width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [58]:
# bars

bars = (
    alt.Chart(
        trump_freq_inauguration,
        title="@realDonaldTrump tweet frequency since inauguration",
    )
    .mark_bar(size=2)
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(grid=False, title="", tickCount=6, format=("%b. %Y")),
        ),
        y=alt.Y(
            "count:Q",
            scale=alt.Scale(domain=(0, 200)),
            axis=alt.Axis(
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=6,
                title="Daily tweet counts and mean",
            ),
        ),
    )
)

# rule showing mean

rule = (
    alt.Chart(trump_freq_inauguration).mark_rule(color="red").encode(y="mean(count):Q")
)

# rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align="center",
    baseline="middle",
    dx=200,
    dy=10,
    fontWeight="bold",
).encode(text=alt.Text("mean(count):Q", format=".2"))

# go
(bars + rule + text).properties(height=400, width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [59]:
heatmap = (
    alt.Chart(
        trump_freq_inauguration,
        title="@realDonaldTrump daily tweet frequency since inauguration",
    )
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color("count:Q", title="Tweet count"),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("count:Q", title="Tweets"),
        ],
        facet=alt.Facet("year(date):O", title="", columns=1),
    )
    .properties(width=600, height=200)
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

alt.Chart(...)

In [60]:
trump_RT_freq_inauguration = trump_tweets_day_prez_df.sort_values(
    ["count"], ascending=False
)

In [61]:
trumpall_prez_rt = trumpall_prez[
    (trumpall_prez["isRetweet"] == True) | (trumpall_prez["isRetweet"] == "t")
]
trumpall_prez_no_rt = trumpall_prez[
    (trumpall_prez["isRetweet"] == False) | (trumpall_prez["isRetweet"] == "f")
]

In [62]:
len(trumpall_prez_rt)

9033

### Retweets

In [63]:
trump_tweets_day_prez_rt = trumpall_prez_rt.groupby(["date"]).size()
trump_tweets_day_prez_rt_df = pd.DataFrame(
    {"date": trump_tweets_day_prez_rt.index, "count": trump_tweets_day_prez_rt.values}
)

In [64]:
trump_tweets_day_prez_rt_df.sort_values(by="date", ascending=False).head()

,date,count
844,2020-11-05,2
843,2020-11-04,2
842,2020-11-03,8
841,2020-11-02,14
840,2020-11-01,3


### No retweets

In [65]:
trump_tweets_day_prez_no_rt = trumpall_prez_no_rt.groupby(["date"]).size()
trump_tweets_day_prez_no_rt_df = pd.DataFrame(
    {
        "date": trump_tweets_day_prez_no_rt.index,
        "count": trump_tweets_day_prez_no_rt.values,
    }
)

In [66]:
# retweets
trump_rt_freq_inauguration = trump_tweets_day_prez_rt_df.sort_values(
    ["count"], ascending=False
)
# no retweets
trump_no_rt_freq_inauguration = trump_tweets_day_prez_no_rt_df.sort_values(
    ["count"], ascending=False
)

In [67]:
trump_tweets_day_prez_rt_df.tail()

,date,count
840,2020-11-01,3
841,2020-11-02,14
842,2020-11-03,8
843,2020-11-04,2
844,2020-11-05,2


In [68]:
### Try a different color scheme

heatmap = (
    alt.Chart(
        trump_no_rt_freq_inauguration,
        title="@realDonaldTrump daily tweet (without RT) frequency since inauguration",
    )
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color(
            "count:Q", title="Tweet count", scale=alt.Scale(scheme="blues")
        ),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("count:Q", title="Tweets"),
        ],
        facet=alt.Facet("year(date):O", title="", columns=1),
    )
    .properties(width=600, height=180)
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

alt.Chart(...)

In [69]:
# bars
bars = (
    alt.Chart(trump_no_rt_freq_inauguration, title="@realDonaldTrump tweet frequency")
    .mark_bar(size=2)
    .encode(
        x=alt.X(
            "date:T", axis=alt.Axis(grid=False, title="", tickCount=6, format=("%B %Y"))
        ),
        y=alt.Y(
            "count:Q",
            scale=alt.Scale(domain=(0, 70)),
            axis=alt.Axis(
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=5,
                title="Daily tweet counts and mean (without retweets) since inauguration",
            ),
        ),
    )
)

# rule showing mean
rule = (
    alt.Chart(trump_no_rt_freq_inauguration)
    .mark_rule(color="red")
    .encode(y="mean(count):Q")
)

# rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align="center",
    baseline="middle",
    dx=50,
    dy=10,
    fontWeight="bold",
).encode(text=alt.Text("mean(count):Q", format=".2"))

# go
(bars + rule + text).properties(height=400, width=620).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [70]:
trumppres[
    (trumppres["text"].str.contains("ban"))
    & (trumppres["text"].str.contains("Twitter"))
]

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,year,month,day,hour,minute,time,user,diff
54097,53528,1124454991341793300,"So surprised to see Conservative thinkers like James Woods banned from Twitter, and Paul Watson banned from Facebook! https://t.co/eHX3Z5CMXb",f,f,Twitter for iPhone,106722,37811,2019-05-03,NaN,2019,5,3,16,25,16:25:35,realDonaldTrump,0.0
54072,53503,1124743267873116200,"How can it be possible that James Woods (and many others), a strong but responsible Conservative Voice, is banned from Twitter? Social Media &amp, Fake News Media, together with their partner, the Democrat Party, have no idea the problems they are causing for themselves. VERY UNFAIR!",f,f,Twitter for iPhone,144190,39310,2019-05-04,NaN,2019,5,4,11,31,11:31:06,realDonaldTrump,0.0
53893,53314,1127188110410952700,RT @nedryun: Reminder: this guy is still on Twitter. Laura Loomer is not. The solution is not to ban this anti-Semite. The goal is to have...,t,f,Twitter for iPhone,0,7588,2019-05-11,NaN,2019,5,11,5,26,05:26:02,realDonaldTrump,0.0
53400,52818,1137702218835136500,"Twitter should let the banned Conservative Voices back onto their platform, without restriction. It’s called Freedom of Speech, remember. You are making a Giant Mistake!",f,f,Twitter for iPhone,157575,45210,2019-06-09,NaN,2019,6,9,5,45,05:45:21,realDonaldTrump,24.0
53378,52796,1138024129616142300,"RT @realDonaldTrump: Twitter should let the banned Conservative Voices back onto their platform, without restriction. It’s called Freedom o...",t,f,Twitter for iPhone,0,45210,2019-06-10,NaN,2019,6,10,3,4,03:04:30,realDonaldTrump,0.0
5919,4996,1316709525525856300,"RT @scrowder: Shortly after the @Facebook Communications director admitted they were shadow banning today's Biden exposé, @Twitter has now…",t,f,Twitter for iPhone,0,10837,2020-10-15,NaN,2020,10,15,4,56,04:56:28,realDonaldTrump,0.0
859,865,1332317394165968899,"Wow! Twitter bans highly respected Pennsylvania State Senator Doug Mastriano after he did a great job of leading a hearing on the 2020 Election fraud. They and the Fake News, working together, want to SILENCE THE TRUTH. Can’t let that happen. This is what Communist countries do!",False,False,Twitter for iPhone,277136,73437,2020-11-27,True,2020,11,27,5,36,05:36:34,realDonaldTrump,0.0
161,163,1344359034716106755,"Twitter is shadow banning like never before. A disgrace that our weak and ineffective political leadership refuses to do anything about Big Tech. They’re either afraid or stupid, nobody really knows!",False,False,Twitter for iPhone,259126,63698,2020-12-30,False,2020,12,30,11,5,11:05:45,realDonaldTrump,0.0


In [71]:
trumppres[trumppres["id"] == "1137702218835136500"][["user", "text"]]

,user,text
53400,realDonaldTrump,"Twitter should let the banned Conservative Voices back onto their platform, without restriction. It’s called Freedom of Speech, remember. You are making a Giant Mistake!"


---